In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib widget
import scipy.stats as stats

# Mock function to simulate reading from CSV files
def read_csv_file(file_path):
    # Simulate reading the CSV file
    # Replace this with pd.read_csv(file_path) in a real scenario
    return pd.read_csv(file_path)

## remember to change also the meal time

day = 'd8' #d8, d12

file = '%s_power_width_norm'%day    #d1_power_abs'   d1_power_width_norm 

# Folder containing the CSV files (replace with your actual folder path)
folder_path = ''

norm = False


# Initialize the main data structure
'''
#Old bands
data = {
    "0-0.2Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "0.2-5Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "5Hz-10Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "10Hz-100Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "100-200Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "200-300Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
}
'''
data = {
    "0-0.05Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "0.05-0.2Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "0.2-1Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "1Hz-5Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "5Hz-300Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
    "300-2000Hz": {"r1": [], "r2": [], "r3": [], "r4": []},
}

# Meal times (these are the period times where meals occur - note indices is 1 less (starts at 0))
meal_times_d1 = {
    "r1": [15,16, 28],
    "r2": [15, 16],
    "r3": [15,21,22,23],
    "r4": [20,21]
}

meal_times_d8 = {
    "r1": [15,16, 28],
    "r2": [15, 16],
    "r3": [15,16, 21,22,23],
    "r4": [20,21]
}

meal_times_d12 = {
    "r1": [15,16, 21, 22],
    "r2": [20, 28],  # 20 very little
    "r3": [21,22, 23],  # from original recording 15,16,17, but I've added 6 min because of the original 6 minutes the rat was already in the cage and just plugged. Not for rat 4 because she was taken out and thne in again.
    "r4": [15, 16]
}


if day=='d1':
    meal_times = meal_times_d1
elif day=='d8':
    meal_times = meal_times_d8
elif day=='d12':
    meal_times = meal_times_d12
    
select_meals = meal_times

# Subtract 1 from each time in the lists for indexing
meal_times = {key: [time - 1 for time in value] for key, value in select_meals.items()}

# Iterate through the CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        # Extract region ID from the filename (e.g., 'r1', 'r2', etc.)
        parts = file_name.split('_')
        region_id = parts[1]
        
        # Read the CSV file
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        
        # Assuming the first column contains frequency bands
        freq_column = df.columns[0]
        
        # Loop through each frequency band we are interested in
        for freq_range in data.keys():
            # Filter the rows where the frequency band matches the desired range
            freq_data = df[df[freq_column] == freq_range]
            
            if not freq_data.empty:
                # Collect the data from the relevant rows and columns
                data_values = freq_data.iloc[:, 1:].values.flatten().tolist()  # Exclude the first column and flatten the values
                
                # Store the data in the corresponding region and frequency band
                data[freq_range][region_id] = data_values




### Merge two bands

In [ ]:
#''' 
#Code to merge first 2 bands:

# Initialize new data structure for merged bands
merged_band_name = "0-0.2Hz"
merged_data = {merged_band_name: {}}

# Initialize empty lists for merged data
for rat in data["0-0.05Hz"]:
    merged_data[merged_band_name][rat] = []

# Sum data from the first two bands for each rat
for rat in data["0-0.05Hz"]:
    merged_data[merged_band_name][rat] = (
        np.array(data["0-0.05Hz"][rat]) + np.array(data["0.05-0.2Hz"][rat])
    ).tolist()

# Create a new dictionary with the merged band first
new_data = {merged_band_name: merged_data[merged_band_name]}

# Add remaining bands except the ones that were merged
for band in data:
    if band not in ["0-0.05Hz", "0.05-0.2Hz"]:
        new_data[band] = data[band]

# Update the original dictionary
data = new_data

## Plot violin all rats in same violin (first 15 min)

In [ ]:

bands = ['0-0.2Hz', '0.2-1Hz', '1Hz-5Hz', '5Hz-300Hz', '300-2000Hz']
rats = ['r1', 'r2', 'r3', 'r4']
markers = ['o', 's', 'D', '^']  # 4 markers for the 4 rats
colors = sns.color_palette('deep', n_colors=len(rats))  # Unique colors for each rat

# Define ylim values for each subplot
ylim_values = {
    '0-0.2Hz': [-20000, 40000],
    '0.2-1Hz': [-11000, 22000],
    '1Hz-5Hz': [-3000, 6000],
    '5Hz-300Hz': [-50, 100],
    '300-2000Hz': [-0.5, 1]
}

# Convert the dictionary to a DataFrame
dataset = []
for band in bands:
    for rat in rats:
        # Select only the first 15 non-NaN points
        values = [v for v in data[band][rat] if not pd.isna(v)][:15]
        dataset.extend([(band, rat, value) for value in values])

df = pd.DataFrame(dataset, columns=['Frequency Band', 'Rat', 'Value'])


# Create subplots: one for each frequency band
fig, axes = plt.subplots(nrows=1, ncols=len(bands), figsize=(20, 8), sharey=False)

for i, band in enumerate(bands):
    ax = axes[i]
    band_data = df[df['Frequency Band'] == band]
    
    sns.violinplot(x='Frequency Band', y='Value', data=band_data, inner=None, ax=ax, palette='pastel')
    #sns.stripplot(x='Frequency Band', y='Value', hue='Rat', data=band_data, dodge=True, jitter=True, ax=ax, palette='Set1', size=10) 
    # Plot each rat with a different marker and color
    for rat, marker, color in zip(rats, markers, colors):
        rat_data = band_data[band_data['Rat'] == rat]
        sns.stripplot(x='Frequency Band', y='Value', data=rat_data,
                      ax=ax, jitter=True, marker=marker, color=color, size=10,
                      edgecolor="black", dodge=True, label=rat)
   
    ax.set_title(f'{band}')
    ax.set_xlabel('')
    ax.set_ylabel('Value' if i == 0 else '')
    ax.legend(title='Rat', loc='upper right')

    # Set ylim based on the defined values
    if band in ylim_values:
        ax.set_ylim(ylim_values[band])

plt.tight_layout()
plt.show()

plt.savefig('%s/violin_open_field_15min_allSame.svg' % (folder_path), dpi=300, bbox_inches='tight', facecolor='w')


## Plot summary first 15 min - box plot per 1 min)

In [ ]:

%matplotlib widget

# Generate an array of strings from '0' to 'len(example_dict) - 1'
periods = [str(i) for i in range(16)]
print(periods)

# Colors for each period
colors = plt.cm.tab10(np.linspace(0, 1, len(periods)))

# Symbols for each rat
symbols = ['o', 's', '^', 'D']

# Number of subplots (one for each band)
num_bands = len(data)
fig, axes = plt.subplots(num_bands, 1, figsize=(15, 30))

# Create empty scatter plots for legend
handles = []
labels = []
for rat_idx, symbol in enumerate(symbols):
    scatter = axes[rat_idx].scatter([], [], color='k', marker=symbol, s=100, edgecolor='white', label=f'Rat {rat_idx + 1}')
    handles.append(scatter)
    labels.append(f'Rat {rat_idx + 1}')
    
# Plotting
for idx, (band, rats) in enumerate(data.items()):
    ax = axes[idx]
    
    # Restructure data for boxplot
    all_periods_data = []
    for period_idx in range(len(periods)):
        period_data = [rats[rat][period_idx] if period_idx < len(rats[rat]) else np.nan for rat in rats]
        period_data_cleaned = [val for val in period_data if not np.isnan(val)]
        all_periods_data.append(period_data_cleaned)
    
    # Creating box plot for each period
    boxplot = ax.boxplot(all_periods_data, positions=np.arange(len(periods)), widths=0.6, patch_artist=True)
    
    # Setting colors for each period
    for patch, period_idx in zip(boxplot['boxes'], range(len(periods))):
        patch.set_facecolor(colors[period_idx])
    
    # Changing the color of the median lines
    for median_line in boxplot['medians']:
        median_line.set_color('white')
        
        # Plotting scatter points when there aren't enough data points for a whisker plot
    for period_idx, period_data in enumerate(all_periods_data):
        if len(period_data) <=4:  # Less than 3 points, plot individual points
            for rat_idx, value in enumerate(period_data):
                ax.scatter(period_idx, value, color=colors[period_idx], marker=symbols[rat_idx], s=100, edgecolor='white', zorder=3)
    ax.set_title(f'{band}')
    ax.set_xticks(np.arange(len(periods)))
    ax.set_xticklabels(periods)
    ax.set_ylabel('Value')
    if norm:
        ax.set_ylim([0,1])

# Set common labels and title
fig.text(0.5, 0.04, 'Time (min)', ha='center', fontsize=14)
fig.text(0.04, 0.5, 'Values', va='center', rotation='vertical', fontsize=14)
fig.suptitle('Open field - %s'%file, fontsize=16)

# Add legend for rat markers
fig.legend(handles, labels, loc='upper right', fontsize=12)


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

plt.savefig('%s/summary_open_field_15min.svg' % (folder_path), dpi=300, bbox_inches='tight', facecolor='w')


## Plot 5 min after meal intake

In [ ]:
# Function to select 10 minutes of data after the first meal time
def select_5min_after_first_meal(data, meal_times):
    selected_data = {}
    for band, rats in data.items():
        selected_data[band] = {}
        for rat, values in rats.items():
            if rat in meal_times and meal_times[rat]:
                start_time = meal_times[rat][0]
                end_time = start_time + 10
                selected_data[band][rat] = values[start_time:end_time]
            else:
                selected_data[band][rat] = []
    return selected_data

# Filter data to only include 10 minutes after the first meal time
filtered_data = select_5min_after_first_meal(data, meal_times)

# Generate an array of strings representing the time points after the first meal
meal_periods = [str(i) for i in range(5)]

# Colors for each period
colors = plt.cm.tab10(np.linspace(0, 1, len(meal_periods)))

# Symbols for each rat
symbols = ['o', 's', '^', 'D']

# Number of subplots (one for each band)
num_bands = len(filtered_data)
fig, axes = plt.subplots(num_bands, 1, figsize=(15, 30))

# Dictionary to track which rats have been plotted
rat_included = {rat: False for rat in symbols}

# Plotting
for idx, (band, rats) in enumerate(filtered_data.items()):
    ax = axes[idx]
    all_periods_data = []
    for period_idx in range(len(meal_periods)):
        period_data = [
            rats[rat][period_idx] if period_idx < len(rats[rat]) else np.nan
            for rat in rats
        ]
        period_data_cleaned = [val for val in period_data if not np.isnan(val)]
        all_periods_data.append(period_data_cleaned)

    # Creating box plot for each period
    boxplot = ax.boxplot(
        all_periods_data,
        positions=np.arange(len(meal_periods)),
        widths=0.6,
        patch_artist=True,
    )

    # Setting colors for each period
    for patch, period_idx in zip(boxplot["boxes"], range(len(meal_periods))):
        patch.set_facecolor(colors[period_idx])

    # Changing the color of the median lines
    for median_line in boxplot["medians"]:
        median_line.set_color("white")

    # Plotting scatter points when there aren't enough data points for a whisker plot
    for rat_idx, rat in enumerate(rats):
        rat_values = rats[rat]
        symbol = symbols[rat_idx]
        for period_idx, value in enumerate(rat_values):
            if not np.isnan(value):
                ax.scatter(
                    period_idx,
                    value,
                    color=colors[period_idx],
                    marker=symbol,
                    s=100,
                    edgecolor="white",
                    zorder=3,
                )
                rat_included[symbol] = True  # Mark this rat as included

    ax.set_title(f"{band}")
    ax.set_xticks(np.arange(len(meal_periods)))
    ax.set_xticklabels(meal_periods, rotation=90)
    ax.set_ylabel("Value")

    # Set y-axis limits based on non-nan values if normalization is active
    if norm:
        non_nan_values = [
            val for sublist in all_periods_data for val in sublist if not np.isnan(val)
        ]
        if non_nan_values:
            ymax = (
                max(non_nan_values) + 0.1 * np.ptp(non_nan_values)
                if len(non_nan_values) > 1
                else max(non_nan_values) + 0.1 * abs(max(non_nan_values))
            )
            ax.set_ylim([0, ymax])

# Set common labels and title
fig.text(0.5, 0.04, "Time (min)", ha="center", fontsize=14)
fig.text(0.04, 0.5, "Values", va="center", rotation="vertical", fontsize=14)
fig.suptitle(
    "After meal - %s" % file,
    fontsize=16,
)

# Add legend for rat markers using only the plotted rats
handles = []
labels = []
for rat_idx, symbol in enumerate(symbols):
    if rat_included[symbol]:
        scatter = plt.scatter(
            [], [], color="k", marker=symbol, s=100, edgecolor="white"
        )
        handles.append(scatter)
        labels.append(f"Rat {rat_idx + 1}")  # Adjust this based on the rat indexing logic

fig.legend(handles, labels, loc="upper right", fontsize=12)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

plt.savefig('%s/summary_after_meal_5min.svg' % (folder_path), dpi=300, bbox_inches='tight', facecolor='w')


## Plot whole duration per rat

In [ ]:
# Periods
# Get the length of the longest value in the dictionary
max_length = max(len(value) for value in data["300-2000Hz"].values())
periods = [str(i) for i in range(max_length)]

# Colors and markers for each rat
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, orange, green, red (color-blind friendly)
symbols = ['o', 's', '^', 'D']  # Different markers for each rat

# Number of subplots (one row per band, one column per rat)
num_bands = len(data)
num_rats = len(next(iter(data.values())))

# Creating subplots
fig, axes = plt.subplots(num_bands, num_rats, figsize=(25, 10), sharex=False, constrained_layout=True)

# Plotting
for row_idx, (band, rats) in enumerate(data.items()):
    for col_idx, (rat, values) in enumerate(rats.items()):
        ax = axes[row_idx, col_idx]
        
        # Scatter plot for each rat with specific color and marker
        ax.scatter(range(len(values)), values, color=colors[col_idx], marker=symbols[col_idx], s=50, edgecolor='black', zorder=3)
        
        # Axis labels and title
        if col_idx == 0:
            ax.set_ylabel(f'{band}\n[unitless]', fontsize=12)
        if row_idx == num_bands - 1:
            ax.set_xlabel('Time (min)', fontsize=12)
        ax.set_xticks(np.arange(len(periods)))
        ax.set_xticklabels(periods, fontsize=10)
        ax.set_title(f'Rat {col_idx + 1}', fontsize=14)
        ax.grid(True, linestyle=':', linewidth=0.5, zorder=0)
        
        # Adding vertical lines for meal times specific to this rat
        if rat in meal_times:
            for meal_time in meal_times[rat]:
                ax.axvline(x=meal_time, color='gray', linestyle='--', linewidth=1.5, zorder=1)
                
        if norm:
            ax.set_ylim([0,1])
        else:
            # Automatically set y-axis limit based on the non-NaN data with some padding
            non_nan_values = [val for val in values if not np.isnan(val)]
            if non_nan_values:
                ymax = max(non_nan_values) + 0.1 * np.ptp(non_nan_values) if len(non_nan_values) > 1 else max(non_nan_values) + 0.1 * abs(max(non_nan_values))
                ax.set_ylim([0, ymax])


# Common title for the figure
fig.suptitle('Scatter Plots of Values for Each Rat and Band Across Periods - %s'%file, fontsize=16, weight='bold')

# Creating a unified legend for all subplots
handles = [plt.Line2D([0], [0], marker=symbol, color='w', markerfacecolor=color, markersize=10, markeredgecolor='black', label=f'Rat {i+1}')
           for i, (color, symbol) in enumerate(zip(colors, symbols))]
fig.legend(handles=handles, loc='upper right', fontsize=12)

# Save the figure with high resolution suitable for publications
plt.savefig('%s/scatter_plots_all.svg' % (folder_path), dpi=300, bbox_inches='tight', facecolor='w')

plt.show()

## Impedances and weigths

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rat = 'rat4'

# Data for each day
if rat=='rat1':
    day1 = np.array([485.9395313, 12.41289648, 115.6563594, 42.36145703, 66.58333594, 79.78739063, 
                     233.7221719, 214.5668438, 245.1942656, 101.4938828, 50.51578516, 57.15841406, 
                     364.7893438, 73.98835938, 75.96063281, 11.96935938, 17.97187109, 18.70008984])
    day8 = np.array([421.945125, 14.27085254, 140.9280469, 55.30731641, 70.7175, 48.06517188, 
                     150.3278906, 147.451625, 251.9900469, 235.4773281, 54.95107813, 66.80249219, 
                     79.64786719, 71.5893125, 44.95150781, 14.47843457, 37.9070625, 20.66440625])
    day12 = np.array([550.680625, 18.09059375, 136.7238125, 52.86771094, 74.7840625, 48.08877734, 
                      164.9225781, 157.632875, 211.2958594, 144.9000938, 42.93749219, 57.29083594, 
                      92.5409375, 73.39000781, 55.96807422, 18.57460156, 43.63576563, 14.95491113])
if rat=='rat2':
    # Data for each day for Rat 2
    day1 = np.array([264.0210625, 562.35575, 233.5063281, 244.0386563, 347.280125, 52.48598828, 
                          33.68796875, 18.08146289, 13.70444824, 11.9737168, 7.912387695, 5.322110352])
    
    day8 = np.array([0.487572113, 0.517255554, 0.093254623, 0.77085675, 0.012846301, 73.37376563, 
                          0.057321369, 16.52502344, 0.060248497, 0.569516785, 6.075919922, 0.978715942])
    
    day12 = np.array([430.5405938, 372.985875, 80.29554688, 35.83074219, 71.00535938, 70.71919531, 
                           60.12675, 27.41206836, 11.15177832, 7.673873047, 5.711387695, 6.096672852])
if rat=='rat3':
    # Data for each day for Rat 3
    day1 = np.array([40.49706641, 20.19349805, 26.78314258, 44.66392969, 32.20751758, 19.99575781, 
                          41.77994531, 40.28339453, 47.77094531, 77.80745313])
    
    day8 = np.array([18.53953125, 51.32169141, 45.28263281, 74.4445625, 141.8354688, 32.81712891, 
                          51.14380078, 17.99620898, 51.04545313, 119.1106484])
    
    day12 = np.array([24.0139082, 36.04597656, 33.76975, 78.05527344, 39.77160938, 32.18002148, 
                           47.60669922, 23.38605469, 53.53273828, 105.0636875])
if rat=='rat4':
    # Data for each day for Rat 4
    day1 = np.array([221.9679219, 442.8359063, 272.9625, 107.501, 187.8613594, 194.9749688, 
                          304.9831563, 36.79672266, 276.4936563])
    
    day8 = np.array([1.061173828, 131.0375547, 52.00175391, 71.02435156, 73.41103906, 70.8066875, 
                          81.26953125, 32.1674707, 55.59882813])
    
    day12 = np.array([53.96892969, 117.2627422, 57.62705859, 46.75229688, 70.64332813, 44.30621094, 
                           54.40837109, 74.23201563, 102.9615547])


# Calculate mean and standard error for each day
mean_day1 = np.mean(day1)
std_err_day1 = np.std(day1) / np.sqrt(len(day1))

mean_day8 = np.mean(day8)
std_err_day8 = np.std(day8) / np.sqrt(len(day8))

mean_day12 = np.mean(day12)
std_err_day12 = np.std(day12) / np.sqrt(len(day12))

# Data for plotting
days = ['Day 1', 'Day 8', 'Day 12']
means = [mean_day1, mean_day8, mean_day12]
std_errs = [std_err_day1, std_err_day8, std_err_day12]

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))

# Bar plot with error bars
ax.bar(days, means, yerr=std_errs, capsize=5, color=['grey', 'grey', 'grey'])

# Scatter plot for individual data points
ax.scatter(np.full(len(day1), 0), day1, color='black', label='Day 1', alpha=0.7)
ax.scatter(np.full(len(day8), 1), day8, color='black', label='Day 8', alpha=0.7)
ax.scatter(np.full(len(day12), 2), day12, color='black', label='Day 12', alpha=0.7)

# Add text in the top-right corner showing the number of channels
ax.text(0.95, 0.95, f'n={len(day1)} channels', ha='right', va='top', 
        transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

# Labels and title
ax.set_xlabel('Days')
ax.set_ylabel('Mean Impedance (kΩ)')
ax.set_title('Mean Impedance with Standard Error over Days')

# Display the plot
plt.show()

# Save the figure with high resolution suitable for publications

plt.savefig('%s/Z_%s.svg' % (folder,rat), dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
import matplotlib.pyplot as plt

# Data
days = [0, 1, 2, 4, 7]
rat1 = [256.6, 247, 256, 264, 268]
rat2 = [273, 285, 286, 282, 289]
rat3 = [293.4, 307, 308, 288, 305]
rat4 = [np.nan, 234, 243, 243, 250]

# Plotting
plt.figure(figsize=(10, 6))

# Color-blind friendly colors and markers
colors = ['#0072B2', '#D55E00', '#009E73', '#CC79A7']

plt.plot(days, rat1, marker='o', linestyle='-', color=colors[0], label='Rat 1')
plt.plot(days, rat2, marker='s', linestyle='-', color=colors[1], label='Rat 2')
plt.plot(days, rat3, marker='D', linestyle='-', color=colors[2], label='Rat 3')
plt.plot(days, rat4, marker='^', linestyle='-', color=colors[3], label='Rat 4')

# Adding labels and title
plt.xlabel('Days')
plt.ylabel('Weight (g)')
plt.title('Weight Over Time')

# Adding a grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Adding legend to identify each rat
plt.legend(title='Rats')

# Displaying the plot
plt.show()
plt.savefig('%s/Weights.svg' % (folder), dpi=300, bbox_inches='tight', facecolor='w')